In [1]:
%load_ext autoreload
%autoreload 2
# add . to module name
import sys
sys.path.append('../src/')

In [2]:
import os
from package import key, storage
from package.osm import osm
from package.logger import Timed, rlog
import pandas as pd
import folium

In [3]:
from package.logger import setup
setup("INFO")

In [4]:
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm.get_osm_path_from_city_id(city_id)

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

osm_reader = osm.new_osm_reader(osm_path)

with Timed.info("Getting OSM graph"):
	nodes, edges = osm.get_graph_for_city_cropped_to_stops(osm_reader, stops_df)


[10:32:46] INFO     Reading stops                                                                   ]8;id=975145;file:///tmp/ipykernel_10801/2164838183.py\2164838183.py]8;;\:]8;id=349336;file:///tmp/ipykernel_10801/2164838183.py#5\5]8;;\

           INFO     Reading stops done (0.24 seconds)                                               ]8;id=600870;file:///tmp/ipykernel_10801/2164838183.py\2164838183.py]8;;\:]8;id=303037;file:///tmp/ipykernel_10801/2164838183.py#5\5]8;;\

           INFO     Downloading OSM data                                                            ]8;id=609801;file:///tmp/ipykernel_10801/2164838183.py\2164838183.py]8;;\:]8;id=196070;file:///tmp/ipykernel_10801/2164838183.py#9\9]8;;\

Downloaded Protobuf data 'Koeln.osm.pbf' (75.38 MB) to:
'/tmp/pyrosm/Koeln.osm.pbf'


[10:32:53] INFO     Getting OSM graph                                                              ]8;id=517692;file:///tmp/ipykernel_10801/2164838183.py\2164838183.py]8;;\:]8;id=654776;file:///tmp/ipykernel_10801/2164838183.py#16\16]8;;\

...

           INFO     60020/1331690 (4.51%) edges remaining                                                 ]8;id=958325;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=35893;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#71\71]8;;\

[10:33:47] INFO     Cropping OSM network to stops done (0.92 seconds)                                     ]8;id=157242;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=632257;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#71\71]8;;\

           INFO     Caching OSM network                                                                   ]8;id=511984;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=269808;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#74\74]8;;\

           INFO     Caching OSM network done (0.29 seconds)                                               ]8;id=701005;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py\osm.py]8;;\:]8;id=844489;file:///home/moritz/dev/uni/mcr-py/notebooks/../src/package/osm/osm.py#74\74]8;;\

           INFO     Getting OSM graph done (54.11 seconds)                                         ]8;id=230362;file:///tmp/ipykernel_10801/2164838183.py\2164838183.py]8;;\:]8;id=184439;file:///tmp/ipykernel_10801/2164838183.py#16\16]8;;\

In [5]:
# edges['lit'] = edges['lit'].apply(lambda x: 1 if x == 'yes' else 0)
# edges['not_lit'] = edges['lit'].apply(lambda x: 0 if x == 1 else 1)
# edges['length_not_lit'] = (edges['length'] * edges['not_lit']).astype(int)

# edges['length'] = (edges['length'] * 1000).astype(int)

# edges['weights'] = '(' + edges['length'].astype(str) + ';' + edges['length_not_lit'].astype(str) + ')'

# edges[['u', 'v', 'weights']].to_csv("../data/mlc_edges.csv", index=False)

### multi modal

In [6]:
nodes["has_bicycle"] = False
nodes.loc[nodes.sample(100).index, "has_bicycle"] = True
nodes = nodes[['id', 'has_bicycle']]

In [7]:
walking_nodes = nodes.copy()
bike_nodes = nodes.copy()

walking_nodes["old_id"] = walking_nodes["id"]
walking_nodes["id"] = "W" + walking_nodes["id"].astype(str)

bike_nodes["old_id"] = bike_nodes["id"]
bike_nodes["id"] = "B" + bike_nodes["id"].astype(str)

In [8]:
edges = edges[["u", "v", "length"]]
walking_edges = edges.copy()
bike_edges = edges.copy()

walking_edges["u"] = "W" + walking_edges["u"].astype(str)
walking_edges["v"] = "W" + walking_edges["v"].astype(str)

bike_edges["u"] = "B" + bike_edges["u"].astype(str)
bike_edges["v"] = "B" + bike_edges["v"].astype(str)

In [9]:
# create edges that go from each bicycle node to each walking node - free floating bicycle system
transfer_edges_values = walking_nodes.apply(
    lambda x: ["B" + str(x.old_id), "W" + str(x.old_id), 0], axis=1
)
transfer_edges = pd.DataFrame(
    transfer_edges_values.tolist(), columns=["u", "v", "length"]
)

In [10]:
avg_walking_speed = 1.4 # m/s
avg_biking_speed = 4.0 # m/s

In [11]:
walking_edges["travel_time"] = walking_edges.length / avg_walking_speed
bike_edges["travel_time"] = bike_edges.length / avg_biking_speed

In [12]:
walking_edges

,u,v,length,travel_time
0,W21063145,W7151289920,7.161,5.115000
1,W7151289920,W10929975,7.575,5.410714
2,W10929975,W10853912,132.615,94.725000
4,W367149,W1672380543,7.730,5.521429
5,W1672380543,W1588847134,11.032,7.880000
...,...,...,...,...
1331671,W11105976044,W11105976043,3.237,2.312143
1331672,W11105943446,W11105943462,13.513,9.652143
1331673,W11105943462,W11105943463,9.101,6.500714
1331674,W11105943463,W11105943464,12.710,9.078571


In [13]:
bike_edges["travel_time_bike"] = bike_edges["travel_time"]

In [14]:
edges = pd.concat([walking_edges, bike_edges, transfer_edges], ignore_index=True)
# fill travel_time for transfer edges and
# travel_time_bike for walking and transfer edges
edges = edges.fillna(0)
# weights = trave_time, travel_time_bike, cost
accuracy = 4
accuracy_multiplier = 10 ** (accuracy - 1)
print(accuracy_multiplier)
edges["weights"] = (
    "("
    + (edges["travel_time"].round(accuracy) * accuracy_multiplier)
    .astype(int)
    .astype(str)
    + ";0)"
)
edges["hidden_weights"] = (
    "("
    + (edges["travel_time_bike"].round(accuracy) * accuracy_multiplier)
    .astype(int)
    .astype(str)
    + ")"
)
edges.head(2)


1000


,u,v,length,travel_time,travel_time_bike,weights,hidden_weights
0,W21063145,W7151289920,7.161,5.115000,0.0,(5115;0),(0)
1,W7151289920,W10929975,7.575,5.410714,0.0,(5410;0),(0)


In [15]:
nodes = pd.concat([walking_nodes, bike_nodes])
nodes.head(2)

,id,has_bicycle,old_id
0,W21063145,False,21063145
1,W7151289920,False,7151289920


In [16]:
node_map = {}
for i, node_id in enumerate(nodes.id.unique()):
	node_map[node_id] = i

nodes["old_id"] = nodes["id"]
nodes["id"] = nodes["id"].map(node_map)
edges["u"] = edges["u"].map(node_map)
edges["v"] = edges["v"].map(node_map)

In [17]:
walking_node_map = {}
for i, node_id in enumerate(walking_nodes.id.unique()):
	walking_node_map[node_id] = i

walking_nodes["old_id"] = walking_nodes["id"]
walking_nodes["id"] = walking_nodes["id"].map(node_map)
walking_edges["u"] = walking_edges["u"].map(node_map)
walking_edges["v"] = walking_edges["v"].map(node_map)

In [18]:
print(nodes.id.isna().sum())
print(edges.u.isna().sum())
print(edges.v.isna().sum())

print(walking_nodes.id.isna().sum())
print(walking_edges.u.isna().sum())
print(walking_edges.v.isna().sum())

0
0
0
0
0
0


In [19]:
edges.sort_values('travel_time_bike')

,u,v,length,travel_time,travel_time_bike,weights,hidden_weights
0,0,1,7.161,5.11500,0.00000,(5115;0),(0)
135494,68638,15454,0.000,0.00000,0.00000,(0;0),(0)
135493,68637,15453,0.000,0.00000,0.00000,(0;0),(0)
135492,68636,15452,0.000,0.00000,0.00000,(0;0),(0)
135491,68635,15451,0.000,0.00000,0.00000,(0;0),(0)
...,...,...,...,...,...,...,...
103428,78087,77808,414.769,103.69225,103.69225,(103692;0),(103692)
112629,100751,77799,414.769,103.69225,103.69225,(103692;0),(103692)
61734,55096,55097,488.963,122.24075,122.24075,(122240;0),(122240)
107677,96725,62234,539.549,134.88725,134.88725,(134887;0),(134887)


In [20]:
edges[["u", "v", "weights", "hidden_weights"]].to_csv(
    "../data/mlc_edges.csv", index=False
)

In [21]:
walking_edges["weights"] = (
    "("
    + (walking_edges["travel_time"].round(accuracy) * accuracy_multiplier)
    .astype(int)
    .astype(str)
    + ")"
)


In [22]:
walking_edges[["u", "v", "weights"]].to_csv(
    "../data/mlc_walking_edges.csv", index=False
)

In [23]:
storage.write_any_dict(
    {"node_map": node_map, "walking_node_map": walking_node_map}, "../data/node_map.pkl"
)


In [24]:
start_node_id = walking_node_map["W21063145"]
start_bags = {
	start_node_id: {
		"node_id": start_node_id,
		"path": [start_node_id],
		"values": [0],
	}
}

In [29]:
import mcr_py
from mcr_py import GraphCache

In [37]:
raw_edges = walking_edges[["u", "v", "weights"]].to_dict("records")

gc = GraphCache()
gc.set_graph(raw_edges)

In [38]:
gc.summary()

In [34]:
bags = mcr_py.run_mlc(gc, 0)

In [35]:
bags

{0: [<PyLabel at 0x7fad3baf2e90>]}

In [ ]:
storage.read_any_dict("jk")

0